In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from multiprocessing import cpu_count
from multiprocessing import Pool
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time

In [44]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# Configure WebDriver
chrome_options = Options()
chrome_options.page_load_strategy = 'eager'  # Waits for minimal loading
driver = webdriver.Chrome(options=chrome_options)

# Set timeout and load URL
url = "https://economictimes.indiatimes.com/markets/stocks/news"
driver.set_page_load_timeout(300)

try:
    driver.get(url)
    driver.execute_script("window.stop();")  # Stop loading after initial content
    print("Page loaded or stopped successfully.")
except Exception as e:
    print("Timeout or error occurred:", str(e))

# driver.quit()


Page loaded or stopped successfully.


In [45]:
#Function to click "Load More" button until it disappears or reaches the limit
def click_load_more(max_clicks=100):
    clicks = 0
    page_links = []
    while clicks < max_clicks:
        try:
            # Wait until the button is clickable
            load_more_button = WebDriverWait(driver, 30).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "autoload_continue"))
            )
            # driver.execute_script("window.scrollTo(0, arguments[0].getBoundingClientRect().top + window.pageYOffset);", load_more_button)
            total_height = driver.execute_script("return document.body.scrollHeight")
            # Calculate the scroll position with the offset
            scroll_position = total_height - 2000
            # Scroll to the calculated position
            driver.execute_script(f"window.scrollTo(0, {scroll_position});")
            time.sleep(2)
            # print(load_more_button)
            html_content = driver.page_source
            soup = BeautifulSoup(html_content, 'html5lib')
            pages = soup.select('div.eachStory>h3>a')
            # print("New:")
            # print(pages)
            for page in pages:
                link = page.get('href', 'No Link')
                if link != 'No Link' and link not in page_links:
                    page_links.append(link)
            ActionChains(driver).move_to_element(load_more_button).click(load_more_button).perform()
            clicks += 1
            print(f"Clicked 'Load More' {clicks} times")
            time.sleep(10)  # Allow time for content to load
        except Exception as e:
            print("No more 'Load More' button or an error occurred:", str(e))
            break
    return driver, page_links

# Click "Load More" button
driver1, page_links = click_load_more(max_clicks=100)
len(page_links)

Clicked 'Load More' 1 times
Clicked 'Load More' 2 times
Clicked 'Load More' 3 times
Clicked 'Load More' 4 times
Clicked 'Load More' 5 times
Clicked 'Load More' 6 times
Clicked 'Load More' 7 times
Clicked 'Load More' 8 times
Clicked 'Load More' 9 times
Clicked 'Load More' 10 times
Clicked 'Load More' 11 times
Clicked 'Load More' 12 times
Clicked 'Load More' 13 times
Clicked 'Load More' 14 times
Clicked 'Load More' 15 times
Clicked 'Load More' 16 times
Clicked 'Load More' 17 times
Clicked 'Load More' 18 times
Clicked 'Load More' 19 times
Clicked 'Load More' 20 times
Clicked 'Load More' 21 times
Clicked 'Load More' 22 times
Clicked 'Load More' 23 times
Clicked 'Load More' 24 times
Clicked 'Load More' 25 times
Clicked 'Load More' 26 times
Clicked 'Load More' 27 times
Clicked 'Load More' 28 times
Clicked 'Load More' 29 times
Clicked 'Load More' 30 times
Clicked 'Load More' 31 times
Clicked 'Load More' 32 times
Clicked 'Load More' 33 times
Clicked 'Load More' 34 times
Clicked 'Load More' 35 

1536

In [46]:
# html_content = driver1.page_source
# soup = BeautifulSoup(html_content, 'html5lib')
# pages = soup.select('div.eachStory>h3>a')
# pages

In [50]:
driver.quit()
page_links_final = page_links.copy()
for i, link in enumerate(page_links[:16]):
    page_links[i] = 'https://economictimes.indiatimes.com' + link
page_links



['https://economictimes.indiatimes.com/markets/stocks/news/stock-market-returns-causing-to-move-away-from-traditionally-safe-assets-study/articleshow/116837918.cms',
 'https://economictimes.indiatimes.com/markets/stocks/news/ahead-of-market-10-things-that-will-decide-stock-market-action-on-wednesday/articleshow/116836724.cms',
 'https://economictimes.indiatimes.com/markets/stocks/news/market-trading-guide-muthoot-finance-hindalco-industries-are-among-4-stock-recommendations-for-wednesday/slideshow/116836449.cms',
 'https://economictimes.indiatimes.com/markets/stocks/news/these-8-stocks-hit-52-week-highs-rallied-up-to-44-in-a-month/slideshow/116836068.cms',
 'https://economictimes.indiatimes.com/markets/stocks/news/us-stocks-open-higher-as-ai-boom-rate-cuts-power-bull-run-in-2024/articleshow/116835658.cms',
 'https://economictimes.indiatimes.com/markets/stocks/news/5-stocks-with-a-pe-ratio-lower-than-their-industry-average/slideshow/116834469.cms',
 'https://economictimes.indiatimes.com

In [93]:
import re
from datetime import datetime
def scrap(link):
    description_page = requests.get(link)
    description = BeautifulSoup(description_page.content, 'html5lib')
    description_head = description.select('h1.artTitle')
    if len(description_head) == 1:
        description_head = description.select('h1.artTitle')[0].get_text(strip=True)
    date_str = description.select('time.jsdtTime')
    combined_datetime = ""
    if len(date_str) == 1:
        date_str = description.select('time.jsdtTime')[0].get_text(strip=True)
        match = re.search(r"(\w+ \d{1,2}, \d{4}), (\d{1,2}:\d{2}:\d{2} (AM|PM))", date_str)
        if match:
            date = match.group(1)
            time = match.group(2)
        else:
            date, time = "", ""
        combined_datetime = datetime.strptime(f"{date} {time}", "%b %d, %Y %I:%M:%S %p")
    summary = description.select('h2.summary')
    if len(summary) == 1:
        summary = description.select('h2.summary')[0].get_text(strip=True)
    description_content = description.select('div.artText')
    # print(description_content)
    # Initialize an empty list to store the filtered content
    filtered_text = []
    # Loop through each "artText m" div
    for div in description_content:
        # Stop processing if "div class=inSideInd show" is encountered
        if div.find_next('div', class_='inSideInd show'):
            break

        # Iterate over the children of the div
        for child in div.descendants:
            # Skip tags we want to exclude
            if child.name in ['i', 'em', 'strong']:
                continue
            if child.name == 'style':
                break
            # Handle <a> tags
            if child.name == 'a':
                # Include only if it has ref='dofollow'
                if child.get('rel') == ['dofollow']:
                    filtered_text.append(child.text.strip())
            # Include other text content directly
            if child.name is None:  # It's a NavigableString
                filtered_text.append(child.text.strip())

    # Join the filtered text into a single string for output
    result = ' '.join(filtered_text)
    unwanted_portion_1 = (
        "Stock Trading Masterclass on Value Investing and Company Valuation By - "
        "The Economic Times, Get Certified By India's Top Business News Brand View Program "
        "Stock Trading Market 104: Options Trading: Kickstart Your F&O Adventure By - Saketh R, "
        "Founder- QuickAlpha, Full Time Options Trader View Program Stock Trading Technical Analysis "
        "for Everyone - Technical Analysis Course By - Abhijit Paul, Technical Research Head, Fund Manager- "
        "ICICI Securities View Program Stock Trading Stock Markets Made Easy By - elearnmarkets, Financial "
        "Education by StockEdge View Program Stock Trading Renko Chart Patterns Made Easy By - Kaushik Akiwatkar, "
        "Derivative Trader and Investor View Program Stock Trading Market 101: An Insight into Trendlines and Momentum "
        "By - Rohit Srivastava, Founder- Indiacharts.com View Program Stock Trading Markets 102: Mastering Sentiment "
        "Indicators for Swing and Positional Trading By - Rohit Srivastava, Founder- Indiacharts.com View Program "
        "Stock Trading Dow Theory Made Easy By - Vishal Mehta, Independent Systematic Trader View Program Stock Trading "
        "Market 103: Mastering Trends with RMI and Techno-Funda Insights By - Rohit Srivastava, Founder- Indiacharts.com "
        "View Program Stock Trading ROC Made Easy: Master Course for ROC Stock Indicator By - Souradeep Dey, Equity and "
        "Commodity Trader, Trainer View Program Stock Trading Heikin Ashi Trading Tactics: Master the Art of Trading "
        "By - Dinesh Nagpal, Full Time Trader, Ichimoku & Trading Psychology Expert View Program Stock Trading RSI Made "
        "Easy: RSI Trading Course By - Souradeep Dey, Equity and Commodity Trader, Trainer View Program Stock Trading "
        "Introduction to Technical Analysis & Candlestick Theory By - Dinesh Nagpal, Full Time Trader, Ichimoku & Trading "
        "Psychology Expert View Program"
    )

    unwanted_portion_2 = "(You can now subscribe to our ETMarkets WhatsApp channel )"

    # Remove the unwanted portions
    result = result.replace(unwanted_portion_1, "").replace(unwanted_portion_2, "")
    # Save or print the result
    return description_head, combined_datetime, summary, result


In [94]:
from multiprocessing import cpu_count
from multiprocessing import Pool
with Pool(cpu_count()) as p:
    results = p.map(scrap, page_links)

In [95]:
len(results)

1536

In [96]:
df = pd.DataFrame(results, columns=['Title', 'Timestamp', 'Synopsis', 'Content'])
df = df.replace('', np.nan)
df = df.dropna()
df

/tmp/ipykernel_72135/628576076.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace('', np.nan)


,Title,Timestamp,Synopsis,Content
0,Stock market returns causing to move away from...,2024-12-31 22:21:00,A research paper by RBI economists indicates t...,Return on stock markets has a weak link with o...
1,Ahead of Market: 10 things that will decide st...,2024-12-31 23:08:00,The Indian market closed 2024 with an 8% annua...,The Indian market ended 2024 with an annual re...
4,"US stocks edge up as AI boom, rate cuts power ...",2024-12-31 21:34:00,Wall Street's major indexes opened higher in t...,Wall Street 's major indexes opened higher in ...
6,Indian benchmarks underperform top peers in 20...,2024-12-31 18:27:00,Indian benchmarks blazed to record highs in th...,"The Nifty 50 and Sensex rose 8.8% and 8.2%, re..."
7,Tech view: Nifty charts indicate bullish count...,2024-12-31 18:02:00,Nifty formed a small positive candle on the da...,A small positive candle was formed on Nifty's ...
...,...,...,...,...
1529,Hot stocks for 2025: Brokerages see 15-30% ups...,2024-12-04 09:18:00,Brokerages are optimistic about JSW Infrastruc...,Brokerages have shared optimistic outlooks on ...
1531,Vedanta shares in focus as Crisil upgrades lon...,2024-12-04 09:41:00,Vedanta share price: Vedanta shares are in foc...,"Vedanta shares are in focus today, December 4,..."
1533,"Stocks to buy today: Axis Bank, Ashok Leyland ...",2024-12-04 09:05:00,The Indian market is likely to consolidate on ...,Indian market is likely to consolidate on Wedn...
1534,Bank stocks up over 1% after Lok Sabha passes ...,2024-12-04 10:26:00,Bank stocks saw an uptick as the Lok Sabha pas...,Bank stocks were trading higher on Wednesday w...


In [98]:
df.to_csv('/mnt/c/Users/ankku/Downloads/economic_times_stocks.csv', index = False)